In [ ]:
import plotly.express as px
import geopandas as gpd
import shapely.geometry
import pandas as pd
import numpy as np
import gzip
import dill

def get_data(data_path):
    """Return loaded data from disk."""
    with gzip.open(data_path, 'rb') as f:
        return dill.load(f)

In [ ]:
geo_df = get_data('data/route_gdf.pkl')

In [ ]:
lats = []
lons = []
names = []
states = []

for feature, name, state in zip(geo_df.geometry, geo_df.route_name, geo_df.state):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        states = np.append(states, [state]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)
        states = np.append(states, None)

fig = px.line_mapbox(lat=lats, lon=lons, hover_name=names,
                     mapbox_style="open-street-map", zoom=4)
fig.show()

In [ ]:
route_marks_df = pd.DataFrame.from_dict({
    'lat': lats,
    'lon': lons,
    'name': names,
    'state': states
})

In [ ]:
# with gzip.open('data/route_marks_df.pkl', 'wb') as f:
#         dill.dump(route_marks_df, f)

In [ ]:
route_marks_df[route_marks_df.state == 'California'].reset_index(drop=True)

In [ ]:
plot_df = route_marks_df[route_marks_df.state == 'California'].reset_index(drop=True)
fig = px.line_mapbox(plot_df, lat='lat', lon='lon', hover_name='name',
                     mapbox_style="open-street-map", zoom=4)
fig.show()

In [ ]:
plot_df